In [205]:
import pandas as pd
import numpy as np
import string 
import json
import pickle
import re
from spellchecker import SpellChecker
from nltk.corpus import stopwords as nltk_stopwords

In [206]:
posts = []
for line in open("ranking_test.jsonl", 'r'):
    posts.append(json.loads(line))
    
all_comments = []
for i in range(len(posts)):
    all_comments.append(pd.DataFrame.from_dict(posts[i], orient='columns'))
    
df = pd.concat(all_comments, ignore_index=True)
df.sample(10)

,text,comments
47951,Reference cycle bug in Rust's scoped threads,{'text': 'Why is this such a big thing? There ...
24391,The Growth of Administrative Staff in Universi...,{'text': 'As a former tenured professor I thou...
27818,I track my boyfriend's movements because I wan...,{'text': 'This reads like something out of The...
14212,"On Linux, 'less' can probably get you owned",{'text': 'This research lcamtuf has been doing...
5956,"Go home Twitter, you're drunk",{'text': 'Sorry but could someone please expla...
39825,Google is working on a new VPN service for use...,{'text': 'There may be some good here.My vpn i...
6146,Stripe – Apple Pay,"{'text': 'So, what is the fallback for users o..."
35292,Show HN: Add “Magic” to Your Business,{'text': 'I think this completely misses the p...
61060,"On undoing, fixing, or removing commits in git",{'text': 'I discovered this link a few weeks a...
19121,Ask HN: Which are the remote-friendly places t...,{'text': 'https:&#x2F;&#x2F;weworkremotely.com...


In [207]:
# оценка комментария (целевой признак)
df['score'] = df['comments'].map(lambda a: a['score'])
# Достать текст из словаря
df['comments'] = df['comments'].map(lambda a: a['text'])

In [208]:
# количество слов в комментарии
df['words_num'] = df['comments'].apply(lambda x: len(x.split()))

In [209]:
def clear_cods(s):
    s = re.sub("&#\S+;", "\'", s) # убираем html кодировки символов
    s = re.sub(r"\xa0", " ", s) # убираем html кодировки символов
    return s

In [210]:
# количество орфографических ошибок
def clear_text(s):
    s = re.sub('http\S+', '', s) # убираем ссылки
    s = re.sub('\S+.com', '', s) # убираем ссылки
    s = re.sub("&#\S+;", "\'", s) # убираем html кодировки символов
    s = re.sub(r"\xa0", " ", s) # убираем html кодировки символов
    s = re.sub("[^A-Za-z \']", " ", s) # оставляем только слова
    return s.split()

spell = SpellChecker()
df['mistakes_num'] = df['comments'].apply(lambda x: len(spell.unknown(clear_text(x))))

In [211]:
# кол-во символов
df['letters_num'] = df['comments'].apply(lambda x: len(clear_cods(x)))

In [212]:
# кол-во стоп-слов и процент стоп-слов
stopwords = set(nltk_stopwords.words('english'))
df['stopwords_num'] = df['comments'].apply(lambda x: len([w for w in str(x).lower().split() if w in stopwords]))
df['stopwords_percent_num'] = df['stopwords_num'] * 100 / df['words_num']

In [213]:
# кол-во слов капсом
df["upper_words_num"] = df['comments'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

In [214]:
# кол-во уникальных слов и процент уникальных слов
df['unique_num'] = df['comments'].apply(lambda x: len(set(str(x).split())))
df['unique_percent_num'] = df['unique_num'] * 100 / df['words_num']

In [215]:
# средняя длина слова в комментарии
df["words_mean_len"] = df['comments'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [216]:
# кол-во знаков препинания
df["punct_num"] = df['comments'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

In [217]:
# Кол-во слов с заглавной буквы
df["first_capital_num"] = df['comments'].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

In [218]:
# кол-во ссылок в комментарии
df['url_num'] = df['comments'].apply(lambda x: len(re.findall(r'http\S+', x)))

In [219]:
df = df.drop(['text', 'comments'], axis=1)

# Добавление оценки позитивности комментария

In [220]:
positive = pd.read_csv('pos_neg_test_features', index_col=0)

In [221]:
df['positive'] = positive['positive']

# Добавление косинусного расстояния между постом и комментарием

In [222]:
cos_sim = []
with open('cos_sim_test.txt', 'r') as f:
    for i in f:
        cos_sim.append(float(i.replace('\n', '')))
cos_sim = np.array(cos_sim)

In [223]:
df['cos_sim'] = cos_sim

* text - текст поста
* comments - текст комментария
* score - рейтинг комментария (0 - самый популярный, 4 - самый не популярный)
* words_num - количество слов в комментарии
* mistakes_num - количество орфографических ошибок в комментарии
* letters_num - количество символов в комментарии
* stopwords_num - количество стоп-слов в комментарии
* stopwords_percent_num - процент стоп-слов в комментарии
* upper_words_num - количество слов, написанных капсом
* unique_num - количество уникальных слов
* unique_percent_num - процент уникальных слов в комментарии
* words_mean_len - средняя длина слов в комментарии
* punct_num - количество пунктуационных знаков
* first_capital_num - количество слов с заглавной буквы
* url_num - количество ссылок в комментарии

# Обработка текста

In [224]:
with open('CatBoost.pkl', 'rb') as f:
    CatBoost = pickle.load(f)

In [225]:
df_test = df.drop('score', axis=1)

In [226]:
predictions = CatBoost.predict_proba(df_test)[:, 1]

# Перевести вероятности в числа (0, 1, 2, 3, 4)

In [229]:
probabilities = np.reshape(predictions, (-1, 5))

In [230]:
num = len(probabilities)
k = [[0]*5 for i in range(num)]
for i in range(num):
    for j in range(4):
        for l in range(1, 5-j):
            if probabilities[i][j] < probabilities[i][j+l]:
                k[i][j] += 1
            else:
                k[i][j+l] += 1

In [231]:
predicted_rank = np.array(k)

In [232]:
predicted_rank_vector = np.reshape(predicted_rank, -1)

# Выгрузка результата

In [233]:
posts = []
for line in open("ranking_test.jsonl", 'r'):
    posts.append(json.loads(line))

In [234]:
num_posts = len(posts)

In [235]:
for i in range(num_posts):
    for j in range(5):
        posts[i]['comments'][j]['score'] = int(predicted_rank[i][j])

In [236]:
with open("ranking_test_answers.jsonl", 'w') as f:
    for item in posts:
        f.write(json.dumps(item) + "\n")

In [237]:
# Сохранение в формате txt только ответы
with open('ranking_test_answers.txt', 'w') as outfile:
    for i in predicted_rank_vector:
        outfile.write(str(i) + '\n')